In [0]:
#working with polynomials
from typing import List
class MyPolynomial:
    def __init__(self, coefficients):
        self.coefficients = coefficients


    def __add__(self, rhs): #P(x) Q(x)
        
        max_len = max(len(self.coefficients), len(rhs.coefficients)) #find max length: P=ax^2+bx, Q = c -> max: 2 
       
        self_coeffs = self.coefficients + [0] * (max_len - len(self.coefficients)) #Padding 0 to array: P=[0,b,a] <- ax^2+bx, Q=[c,0,0] <- c
        rhs_coeffs = rhs.coefficients + [0] * (max_len - len(rhs.coefficients))

        result_coeffs=[a + b for a, b in zip(self_coeffs, rhs_coeffs)] #zip same size array into another result array
        while len(result_coeffs) > 1 and result_coeffs[-1] == 0:
            result_coeffs.pop()                 #result: pop unnecessary 0 at the end: [1,2,0] ->[1,2]
        return MyPolynomial(result_coeffs)

    def __mul__(self, rhs):
        
        result_degree = len(self.coefficients) + len(rhs.coefficients) - 1
        result_coeffs = [0] * result_degree
        for i, a in enumerate(self.coefficients):
            for j, b in enumerate(rhs.coefficients):
                result_coeffs[i + j] += a * b
        while len(result_coeffs) > 1 and result_coeffs[-1] == 0:
            result_coeffs.pop()
        return MyPolynomial(result_coeffs)

    def __divmod__(self, divisor):
       
        dividend=MyPolynomial(self.coefficients[:]) 
        divisor=MyPolynomial(divisor.coefficients[:])
        dividend_coeffs =dividend.coefficients
        divisor_coeffs =divisor.coefficients
        #quotient = [0] * max(len(dividend_coeffs) - len(divisor_coeffs), 0)
        quotient = [0]
        while len(dividend_coeffs) >= len(divisor_coeffs):
            
            main_term = dividend_coeffs[-1] / divisor_coeffs[-1]
            degree_diff = len(dividend_coeffs) - len(divisor_coeffs)
            quotient.insert(0, main_term)

            
            for i in range(len(divisor_coeffs)):
                dividend_coeffs[degree_diff + i] -= main_term * divisor_coeffs[i]

           
            while dividend_coeffs and dividend_coeffs[-1] == 0:
                dividend_coeffs.pop()
        remainder=dividend_coeffs or [0]
        return MyPolynomial(quotient), MyPolynomial(remainder)

    def __floordiv__(self, divisor):
          
        q, _ = self.__divmod__(divisor)
        return q

    def __mod__(self, divisor):
       
        _, r = self.__divmod__(divisor)
        return r

    def degree(self):
           return len(self.coefficients) - 1 if self.coefficients else -1
       
    def __repr__(self):
        
        result = []
        for power, coef in enumerate(self.coefficients):
            
            if power == 0:
                result.append(f"{coef}")  
            elif coef == 0:
                continue
            elif power == 1:
                result.append(f"{coef}x")  
            else:
                 result.append(f"{coef}x^{power}")  # Higher degree terms

        return " + ".join(result).replace("+ -", "- ")
##########################################

#Example
polynomial1= MyPolynomial([2,-2,1])
polynomial2= MyPolynomial( [1])

print(polynomial1.degree()) 

print(polynomial1)
print(polynomial1.__repr__())
#Addition
print(polynomial1.__add__(polynomial2))
#Multiplication
print(polynomial1.__mul__(polynomial2))
#Division
quotient, remainder = polynomial1.__divmod__(polynomial2) 

print("Quotient:", quotient)   
print("Remainder:", remainder) 


     



In [0]:
def horners_method(f: MyPolynomial, c):
    result = 0   
    for coeff in reversed(f.coefficients):
        result = result * c + coeff
        
    return result 
#######
#Example
horners_method(polynomial1,3)

In [0]:
def lagrange_interpolation(inputs, outputs) -> MyPolynomial:
   
    n = len(inputs)
    result = MyPolynomial([0]) 

    for i in range(n):
        ai, bi = inputs[i], outputs[i]   
        li = MyPolynomial([1])  
        for j in range(n):
            if i != j:
                aj = inputs[j]
                ci = MyPolynomial([-aj, 1])  
                li =li * ci
                li = MyPolynomial([coeff / (ai - aj) for coeff in li.coefficients])
        tem_polynomial = MyPolynomial([coeff * bi for coeff in li.coefficients])
        result = result + tem_polynomial

    return result
 #######
#Example
inputs=[1,3,5]
outputs=[0,-26,-124]
lagrange_interpolation(inputs, outputs)

In [0]:
def search_primitive_element(q):
    order = q - 1
    divs = divisors(order)
    divs.remove(order) 
    for g in range(2, q-2):  
        if pow(g, order, q) ==1:
            is_primitive = True
            for d in divs:
                if pow(g, d, q) == 1:  
                    is_primitive = False
                    break
            if is_primitive:
                return g
#######
#Example
search_primitive_element(11)              
#print(GF(11).primitive_element())

In [0]:
def chien_search(f: MyPolynomial,q) -> List[int]:
    roots = []
    g = search_primitive_element(q)
    for k in range(1, q-1):
        c= pow(g, k, q)  
        result =  horners_method(f, c) % q
        print(f"g^{k} = {c}, f(g^{k}) = {result}")
        if result  == 0:   
            roots.append(c)

    if not roots:
        print("No roots ")
    return roots
#######
#Example
chien_search(polynomial1,5)